# Using the Odata production environment

## Prerequisites

The production environment is hosted on Google Kubernetes Engine, you will need relevant permissions to access it

#### Install Helm client in the correct version

```
bash apps_travis_script.sh install_helm
```

## Switch to the odata environment

In [ ]:
source switch_environment.sh odata

#### Verify you are connected to the right environment

In [ ]:
cat environments/${K8S_ENVIRONMENT_NAME}/.env
kubectl config get-contexts `kubectl config current-context`

#### Get pods

In [ ]:
kubectl get pods

## Deploying infrastructure changes

After making some changes to infrastructure, you should pull, commit and push, then run the deployment procedure

#### Dry run

In [ ]:
./helm_upgrade_external_chart.sh odata --debug --dry-run

#### Deploy

In [41]:
./helm_upgrade_external_chart.sh odata

hasadna-general-hasadna-cluster-odata-blue$ Switched to context "gke_hasadna-general_europe-west1-b_hasadna-cluster".
Connected to hasadna-general-hasadna-cluster-odata-blue
RELEASE_NAME=odata-odata-odata-blue
CHART_DIRECTORY=charts-external/odata
Release "odata-odata-odata-blue" has been upgraded. Happy Helming!
LAST DEPLOYED: Thu Oct 11 13:06:43 2018
NAMESPACE: odata-blue
STATUS: DEPLOYED

RESOURCES:
==> v1/ConfigMap
NAME   DATA  AGE
ckan   2     9d
nginx  1     9d

==> v1/Service
NAME          TYPE       CLUSTER-IP     EXTERNAL-IP  PORT(S)   AGE
ckan-jobs-db  ClusterIP  10.23.253.50   <none>       5432/TCP  9d
ckan          ClusterIP  10.23.244.86   <none>       5000/TCP  9d
datapusher    ClusterIP  10.23.246.172  <none>       8800/TCP  9d
datastore-db  ClusterIP  10.23.255.146  <none>       5432/TCP  9d
db            ClusterIP  10.23.241.7    <none>       5432/TCP  9d
nginx         ClusterIP  10.23.246.225  <none>       80/TCP    9d
pipelines     ClusterIP  10.23.255.149  <none>   

: 1

#### Wait for pods to start

In [53]:
kubectl get pods

hasadna-general-hasadna-cluster-odata-blue$ NAME                            READY     STATUS    RESTARTS   AGE
ckan-5fd44cf7f8-52d2p           1/1       Running   0          17m
ckan-5fd44cf7f8-gt7cn           1/1       Running   0          17m
ckan-jobs-645758bbf8-mxs2r      1/1       Running   0          16m
ckan-jobs-db-6cc4978758-mb7gk   1/1       Running   0          7d
datastore-db-944bfbc74-2nc7b    2/2       Running   0          7d
db-7dd99b8547-vpf57             2/2       Running   0          9d
nfs-f464599df-sbc87             1/1       Running   0          9d
nginx-66864dc7d9-rlrmj          1/1       Running   0          7d
nginx-66864dc7d9-rrsf5          1/1       Running   0          7d
pipelines-7d99478bcd-6qr5v      1/1       Running   0          29m
redis-ffb5675bb-6qn5c           1/1       Running   0          7d
solr-fc9666c75-jjpgg            1/1       Running   0          9d


: 1

## Manage the CKAN pipelines server

#### Start port forward to the pipelines dashboard

In [57]:
kubectl port-forward $(./kubectl.sh get-pod-name pipelines) 5050

hasadna-general-hasadna-cluster-odata-blue$ Forwarding from 127.0.0.1:5050 -> 5050
Handling connection for 5050
Handling connection for 5050
E1011 13:25:34.700573   21213 portforward.go:303] error copying from remote stream to local connection: readfrom tcp4 127.0.0.1:5050->127.0.0.1:44934: write tcp4 127.0.0.1:5050->127.0.0.1:44934: write: broken pipe
Handling connection for 5050
Handling connection for 5050
E1011 13:26:13.078387   21213 portforward.go:271] error creating error stream for port 5050 -> 5050: Timeout occured
Handling connection for 5050
Handling connection for 5050
Handling connection for 5050
Handling connection for 5050
E1011 13:32:47.843489   21213 portforward.go:271] error creating error stream for port 5050 -> 5050: Timeout occured
Handling connection for 5050
Handling connection for 5050
Handling connection for 5050
E1011 13:34:21.715182   21213 portforward.go:271] error creating error stream for port 5050 -> 5050: Timeout occured
Handling connection for 5050
Hand

Pipelines dashboard should be available at http://localhost:5050

#### Run a pipeline manually

In [56]:
./kubectl.sh exec pipelines -- dpp run --verbose ./ckanext-odata_org_il/update_foi_offices_resource

hasadna-general-hasadna-cluster-odata-blue$ [./ckanext-odata_org_il/update_foi_offices_resource:T_0] >>> INFO    :8e89a28e RUNNING ./ckanext-odata_org_il/update_foi_offices_resource
[./ckanext-odata_org_il/update_foi_offices_resource:T_0] >>> INFO    :8e89a28e Collecting dependencies
[./ckanext-odata_org_il/update_foi_offices_resource:T_0] >>> INFO    :8e89a28e Running async task
[./ckanext-odata_org_il/update_foi_offices_resource:T_0] >>> INFO    :8e89a28e Waiting for completion
[./ckanext-odata_org_il/update_foi_offices_resource:T_0] >>> INFO    :8e89a28e Async task starting
[./ckanext-odata_org_il/update_foi_offices_resource:T_0] >>> INFO    :8e89a28e Searching for existing caches
[./ckanext-odata_org_il/update_foi_offices_resource:T_0] >>> INFO    :8e89a28e Building process chain:
[./ckanext-odata_org_il/update_foi_offices_resource:T_0] >>> INFO    :- ckanext.load_resource
[./ckanext-odata_org_il/update_foi_offices_resource:T_0] >>> INFO    :- ckanext.load_resource
[./ckanext-odata

: 1

#### Run a CKAN request from the pipelines server

In [ ]:
kubectl exec $(./kubectl.sh get-pod-name pipelines) -- bash -c \
    'wget -qO - $CKAN_URL/pipelines/config/odata_org_il --header "Authorization: $CKAN_API_KEY"'

## Create a fresh DB backup

In [58]:
source switch_environment.sh odata
./kubectl.sh exec db -c db -- bash /db-scripts/backup.sh

hasadna-general-hasadna-cluster-odata-blue$ Switching to odata  environment
K8S_ENVIRONMENT_CONTEXT=gke_hasadna-general_europe-west1-b_hasadna-cluster
Switched to context "gke_hasadna-general_europe-west1-b_hasadna-cluster".
Connected to hasadna-general-hasadna-cluster-odata-blue
hasadna-general-hasadna-cluster-odata-blue$ 

: 1

#### Wait a few minutes, backup should be ready, verify using gsutil

In [70]:
gsutil ls -p hasadna-odata -l \
    `kubectl exec $(./kubectl.sh get-pod-name db) -c db-ops \
        -- bash -c 'echo $BACKUP_TARGET_PREFIX$(date +%Y-%m-%d)$BACKUP_TARGET_SUFFIX'`

hasadna-general-hasadna-cluster-odata-blue$    9139027  2018-10-11T13:53:05Z  gs://odata-k8s-backups/production/blue/ckan-db-dump-2018-10-11.ckan.dump
TOTAL: 1 objects, 9139027 bytes (8.72 MiB)


: 1

Deploy an environment from backup - script automatically takes latest today's backup